In [ ]:
from pyspark.sql.session import SparkSession

spark = SparkSession \
    .builder \
    .config("spark.jars",
            "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
    .master("local[*]") \
    .appName("leetcode-problems") \
    .getOrCreate()

In [ ]:
spark.sql('''
with main_cte as (
select
    polls.user_id,
    polls.poll_id,
    polls.poll_option_id,
    poll_answers.correct_option_id,
    polls.amount,
    case
        when poll_option_id = correct_option_id then 'Yes'
    else 'No' end as correct_or_not
from
    polls
left join
    poll_answers
on
    polls.poll_id = poll_answers.poll_id),
total_incorrect_sum as (
select
    poll_id,
    sum(amount) as total_sum
from
    main_cte
where
    correct_or_not = 'No'
group by poll_id),
total_correct_sum as (
select
    poll_id,
    sum(amount) as total_correct_sum
from
    main_cte
where
    correct_or_not = 'Yes'
group by poll_id)
select
    main_cte.user_id,
    main_cte.poll_id,
    round(total_incorrect_sum.total_sum*(amount/total_correct_sum),0) as testing2
from
    main_cte
inner join
    total_incorrect_sum
on
    main_cte.poll_id = total_incorrect_sum.poll_id
inner join
    total_correct_sum
on
    main_cte.poll_id = total_correct_sum.poll_id
where
    correct_or_not = 'Yes';
''')